In [8]:
from PIL import Image
import os
import base64
import io

input_folder = "images"
output_folder = "output"

os.makedirs(output_folder, exist_ok=True)

def convert_to_webp(input_path, output_path):
    with Image.open(input_path) as img:
        img.save(output_path, "WEBP")

def generate_lqip_base64(image_path, output_size=(20, 20)):
    img = Image.open(image_path).convert("RGB")
    img.thumbnail(output_size)
    buffer = io.BytesIO()
    img.save(buffer, format="WEBP")
    buffer.seek(0)
    img_base64 = base64.b64encode(buffer.read()).decode("utf-8")
    return f"data:image/webp;base64,{img_base64}"

for filename in os.listdir(input_folder):
    if filename.lower().endswith(".png"):
        input_path = os.path.join(input_folder, filename)
        output_filename = os.path.splitext(filename)[0] + ".webp"
        output_path = os.path.join(output_folder, output_filename)

        convert_to_webp(input_path, output_path)
        base64_placeholder = generate_lqip_base64(output_path)

        html_tag = f'''
<img class="lazy-img"
     src="{base64_placeholder}"
     data-src="output/{output_filename}"
     alt="{filename}"
     loading="lazy">
'''
        print(html_tag)

# add to  the CSS:
```css
.lazy-img {
    filter: blur(10px);
    transition: filter 0.1s ease;
}
```
# add to JS:
```js
document.addEventListener("DOMContentLoaded", () => {
  document.querySelectorAll(".lazy-img").forEach(img => {
    const realSrc = img.dataset.src;
    const newImg = new Image();
    newImg.src = realSrc;
    newImg.onload = () => {
      img.src = realSrc;
      img.style.filter = "none";
    };
  });
});
```